In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals
from __future__ import print_function

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from nltk.util import ngrams
import string
import re

from sklearn.mixture import BayesianGaussianMixture
from sklearn.neighbors import NearestNeighbors

In [70]:
# # Automate creation of gold data and gold matrix from 5 unique forum topics.

# # Useful topics

# # 5705e19d81e07b055c000bc0 - Same scriptures for bibles? - Please offer your reflections on the fact that not all Bibles have the same contents. Do you think it is important that people belonging to the same tradition share the same scriptures? Why or why not? Please post your response and respond to the posts of at least two of your peers.
# # 5702af2c10e0fd052100099c - Christianity practiced locally - Post what you found along with brief comments about what it is and why you find it distinctive or typical of Christianity where you live. Read at least five postings of your peers in preparation for answering the next discussion question.
# # 570bf4976fcfa50548000266 - Beginnings of the gospels - Now that you’ve read the introductions to the four New Testament gospels and examined their particular similarities, differences, and emphases, consider the following questions: What do readers learn about the content of these gospels and about Jesus by reading their introductions? What expectations might readers have as they begin to read each gospel? How would you now address the issue of why the New Testament has four gospels rather than a single narrative? Please post your reflections on the discussion board and respond to at least two of your peers.
# # 5705e53810e0fd052d000c3a - Canons - Often people ask about who chose which books would be included in the Bible and why. We’ve asked you to notice that the content of Bibles are different and changing. The Protestant Luther, for example, removed books from the Catholic canon and created a smaller Protestant canon. Ethiopians have a much larger canon. And now as more early Christian books are discovered in Egypt, some are suggesting these might be added to a “new New Testament.” Please share 1-3 things about the readings on canon and new discoveries that you found especially interesting or surprising. Please post your reflections and respond to the posts of at least two of your peers.
# # 570bf5946fcfa5055000026e - Nativity Scene - The nativity scene of Jesus’s birth is widely represented not only in artistic representations, but today also by small models in homes, life-size exhibits at churches, or through live staged enactments—and indeed you may have seen them in your own home, church, or another public space. If we consider these representations as interpretations of the gospel narratives, what do you think they communicate about Jesus? About the beliefs of Christians? What difference do you think it makes whether people read each gospel independently, read and compare all the gospel birth stories, or interpret Jesus’s birth through harmonized nativity scenes? Please post your reflections on the discussion board and respond to at least two of your peers. 

# topics = [
#   "5705e19d81e07b055c000bc0", 
#   "5702af2c10e0fd052100099c", 
#   "570bf4976fcfa50548000266", 
#   "5705e53810e0fd052d000c3a", 
#   "570bf5946fcfa5055000026e",
# ]

# df_gold = df[df.thread_id_from_url.isin(topics)]
# topic_map = dict(zip(topics, range(len(topics))))
# df_gold["topic_cluster"] = df_gold.thread_id_from_url.apply(lambda x: topic_map[x])

# from scipy.spatial.distance import pdist, squareform

# gold_matrix = squareform(pdist(pd.DataFrame(df_gold.topic_cluster)))
# for row in range(len(gold_matrix)):
#   for col in range(len(gold_matrix)):
#     if gold_matrix[row][col] == 0:
#       gold_matrix[row][col] = 1
#     else:
#       gold_matrix[row][col] = 0
      
# # pickle.dump( gold_matrix, open( "gold_matrix_for_HarvardX__HDS_3221_2X__1T2016.p", "wb" ) )
# # pickle.dump( df_gold, open( "gold_data_HarvardX__HDS_3221_2X__1T2016.p", "wb" ) )

# # Write to file
# cols = ['thread_id_from_url', 'body', 'num_replies', 'body_length', 'sum_body_length_per_topic', 'num_responses', 'avg_response_length', 'topic_cluster']
# pd.DataFrame(gold_matrix).to_csv("gold_matrix_for_HarvardX__HDS_3221_2X__1T2016.csv.gz", compression='gzip', index=False)
# pd.DataFrame(df_gold, columns = cols).to_csv("gold_data_HarvardX__HDS_3221_2X__1T2016.csv.gz", compression='gzip', index=False)

In [3]:
pd.set_option('max_colwidth',5000)

In [4]:
def remove_punctuation(text):
    return re.sub(ur"\p{P}+", "", text)

In [16]:
def genWord2VecCommentEmbedding(w2v_model, comments_array, bigrams=True, verbose=True):
    '''Returns a matrix, each row is an embedding for a comment, each column is a dimension. Order is
 preserved.'''
 
    #First pass of TFIDF vectorizer to generate vocabulary of comments.
    #vec = HashingVectorizer(ngram_range=(1, 2), stop_words='english')
    vec = TfidfVectorizer(stop_words='english')
    C = vec.fit_transform(comments_array).toarray()
    
    if verbose:
        print("First TFIDF pass complete. Reducing comment vocabulary based on w2v table")
        
    #vocab = [i.replace(" ", "_") for i in vec.get_feature_names()]
    if bigrams:
    
        #This method computes all unique bigrams from our comments and adds them to the vocabulary
        punc = string.punctuation
        bigrams = []
        for c in comments_array:
            bigrams += [i[0] + "_" + i[1] for i in ngrams(remove_punctuation(unicode(c, 'utf-8')).split(), 2)]
        unique_bigrams = list(set(bigrams))
        vocab = vec.get_feature_names() + unique_bigrams
        
        #This method for computing bigrams finds all the bigrams which already exist in our w2v model.
        #Only add word2vec bigrams if they are in our vocab list
        #w2v_bigrams = [i for i in w2v_model.index \\
        #               if "_" in i.decode("UTF-8") and \\
        #               pd.Series(i.decode("UTF-8").split("_")).isin(vec.get_feature_names()).all()]
        #vocab = vec.get_feature_names() + w2v_bigrams
    else:
        vocab = vec.get_feature_names()
        
    W = w2v_model[w2v_model.index.isin(vocab)].sort()
    
    #Second pass of TFIDF vectorizer using vocabulary in common with w2v model and comments
    vec = TfidfVectorizer(vocabulary=W.index)
    C = vec.fit_transform(comments_array).toarray()
    W = W.values
    
    A = np.dot(C, W)
    if verbose:
        print("Shape of embedding comments matrix:",np.shape(A))
        
    return A

In [6]:
# w2v_matrix = pickle.load( open( "/Users/cgn/Dropbox (MIT)/cgn/bq/w2v_matrix.p", "rb" ) )
# vocab = np.array(pickle.load( open( "/Users/cgn/Dropbox (MIT)/cgn/bq/word.p", "rb" ) ) )
w2v_matrix = pickle.load( open( "w2v_matrix_for_HarvardX__HDS_3221_2X__1T2016.p", "rb" ) )
vocab = np.array(pickle.load( open( "vocab_for_HarvardX__HDS_3221_2X__1T2016.p", "rb" ) ) )

In [4]:
np.shape(w2v_matrix)

(3000000, 300)

In [8]:
df = pd.read_csv('HarvardX__HDS_3221_2X__1T2016_scored_forum_responses_and_features.csv.gz', compression='gzip')
topic_id = '5705e20881e07b74f500019d'
X = df[df.thread_id_from_url==topic_id].body.values

In [9]:
# Creation of reduced w2v_matrix model with 1-gram through 6-gram

# all_txt = " ".join(list(df.body.apply(lambda x: unicode(x, 'utf-8'))))
# translate_table = dict((ord(char), None) for char in string.punctuation)   
# all_words = all_txt.translate(translate_table).split()

# ngram_start = 1
# ngram_end = 6
# all_words_ngram = ["_".join(all_words[i:i + ngram]) for ngram in range(ngram_start, ngram_end +1) for i in range(len(all_words) - (ngram - 1))]
# unique_words_list = list(set(all_words_ngram))

# # Build hash map from word to index
# vocab_map = dict(zip(vocab, range(len(vocab))))

# # Get indices of items in vocab that are contained in unique_words_list
# idx_with_Nones = np.array([vocab_map.get(word) for word in unique_words_list])
# idx = [i for i in idx_with_Nones if i is not None]

# # Reduce w2v_matrix size by limiting to all unique words in this course.
# vocab = vocab[idx]
# w2v_matrix = w2v_matrix[idx]

# pickle.dump( w2v_matrix, open( "w2v_matrix_for_HarvardX__HDS_3221_2X__1T2016.p", "wb" ) )
# pickle.dump( vocab, open( "vocab_for_HarvardX__HDS_3221_2X__1T2016.p", "wb" ) )

In [10]:
# Find the largest n_gram.
a = [i for i in vocab if "__" not in i]
index_of_max = np.argmax([len(i.split("_")) for i in a])
a[index_of_max]

In [17]:
comment_embeddings = genWord2VecCommentEmbedding(pd.DataFrame(w2v_matrix, index=vocab), X)

First TFIDF pass complete. Reducing comment vocabulary based on w2v table
Shape of embedding comments matrix: (162, 300)


/Users/cgn/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:33: FutureWarning: sort(....) is deprecated, use sort_index(.....)


In [18]:
# Number of clusters
k = 5

# Number of neighbors in NN model
n_neighbors = 3

# Init Nearest Neighbors model
nn = NearestNeighbors()
nn.fit(comment_embeddings)

# Init DPGMM model
dpgmm = BayesianGaussianMixture(weight_concentration_prior_type="dirichlet_process", n_components=k)
dpgmm.fit(comment_embeddings)

# Get clusters from comments
#clusters = dpgmm.predict(comment_embeddings)

# Find the nearest comment to each cluster mean 
mean_comments = df.ix[[nn.kneighbors(X=i, n_neighbors=1, return_distance=False)[0][0] for i in dpgmm.means_]]
mean_comments['cluster'] = range(k)
mean_comments

/Users/cgn/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/cgn/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/cgn/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  

,thread_id_from_url,body,num_replies,body_length,sum_body_length_per_topic,num_responses,avg_response_length,cluster
38,571fd18a7e622b052d0002eb,learned about arts expression of religion cultures and social interpretations and the many different catagorization of the bibles diverse publications all have completely opened my way to want to travel and capture a moment of past history and write about it.,0,260,11889,35,339.685714,0
112,5702b32fc3bac7052f000a0c,"I didn't learn anything new, but the summary is good!",0,54,50248,141,356.368794,1
9,571fd18a7e622b052d0002eb,"It is fascinating to get glimpse of the ways in which Christianity has shaped the Western concepts of time and place. These are so central to those of us that live in this culture, that it is really difficult to imagine how these constructs might have evolved differently. I also find it difficult to envision the future of Christianity, which is likely to incorporate a significant influence from non-Western cultures, and may change in ways that will seem strange to my Western sensibilities.",3,496,11889,35,339.685714,2
65,5702b32fc3bac7052f000a0c,"This has been a good refresher, but I'm looking forward to more detail later on in the course.",1,94,50248,141,356.368794,3
152,5702b32fc3bac7052f000a0c,Very surprised and gratified to see my Tradition (LDS) given such prominence. I'm loving seeing the different approaches that people are taking.,0,144,50248,141,356.368794,4


In [157]:
kneighbors = [nn.kneighbors(X=i, n_neighbors=n_neighbors, return_distance=False)[0] for i in dpgmm.means_]
kneighbors = df.ix[[x for i in kneighbors for x in i]]
kneighbors["cluster"] = [i for i in range(k) for _ in range(n_neighbors)]
opinion_summaries = ["Bibles are different, should view all", "Diff cultures have diff books. Its okay.", "Bibles are diff because of human authorship", "Bibles are different and its okay to use different bibles.","equivocating both sides"]
kneighbors["opinion"] = [i for i in opinion_summaries for _ in range(n_neighbors)]
kneighbors

/Users/cgn/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/cgn/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/cgn/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  

,thread_id_from_url,body,num_replies,body_length,sum_body_length_per_topic,num_responses,avg_response_length,cluster,opinion
38,571fd18a7e622b052d0002eb,learned about arts expression of religion cultures and social interpretations and the many different catagorization of the bibles diverse publications all have completely opened my way to want to travel and capture a moment of past history and write about it.,0,260,11889,35,339.685714,0,"Bibles are different, should view all"
122,5702b32fc3bac7052f000a0c,Nothing new. Probably thinking on Christianity in different environments and cultures. What is common in al the forms. Not that challenging yet.,0,144,50248,141,356.368794,0,"Bibles are different, should view all"
74,5702b32fc3bac7052f000a0c,"The view of Lutherans, that God's grace is promised not earned. The Pentacostal beliefs in the gifts othe Holy Spirit. The third being the ironoy of the splits in the Catholic church, between Roman Catholics and Eastetn Orthodox.",1,230,50248,141,356.368794,0,"Bibles are different, should view all"
65,5702b32fc3bac7052f000a0c,"This has been a good refresher, but I'm looking forward to more detail later on in the course.",1,94,50248,141,356.368794,1,Diff cultures have diff books. Its okay.
126,5702b32fc3bac7052f000a0c,The ex-communication of Martin Luther and the beginning of the Protestant Church. Interesting stuff on the Mormons as well.,0,124,50248,141,356.368794,1,Diff cultures have diff books. Its okay.
48,5702b32fc3bac7052f000a0c,"1) The schism of the ortodox church was interesting. \n2) Reading about the ethiopian church was very fascinating. \n3) It gave me light to compare how different cultures live their christianity and made me wonder ""what is the truth?""",4,232,50248,141,356.368794,1,Diff cultures have diff books. Its okay.
77,5702b32fc3bac7052f000a0c,"Three comments as opposed to what I learned:\n\n1. it was at a high level and most of it I had heard at one time or another before. On the other hand it was a nice summary that showed me just how much there is to learn if the class dives deeper into scripture and the difference between the different religions\n2. I am interested in how countries that are primarily Christian have done bad things to other countries and to their own people. This is not to drag up old history, or portion out blame, but to better understand how/why it seems to be consistent that ""good people do bad things to other people"".\n3. I am interested how scripture deals with the question of evil. I have always had a problem with the idea of how a benevolent God correlates to a world with a great deal of evil.\n\nAgain these questions are not to apportion blame or challenge other people's beliefs, they are honest questions of how scripture deals with the world as it is. I can see how the idea of culture affecting religion, and how it is important to understand stiltuatedness fits into some of these questions.",0,1094,50248,141,356.368794,2,Bibles are diff because of human authorship
120,5702b32fc3bac7052f000a0c,"In this first day - I learned that there was an original Ethiopian Christian sect, just how vastly each culture changed the focus of Christianity over time and place, and how the latest big expressions are coming from Asia and South America.",0,241,50248,141,356.368794,2,Bibles are diff because of human authorship
31,571fd18a7e622b052d0002eb,"I never knew the story of 'Our Lady of Gaudalupe' though I am familiar with its existence and a few local churches in my area. The art was fascinating and very thought provoking! Reading and in some cases, re-reading passages, allowed me to make a connection and have a better understanding of different protestant practices and where they were born out of. I think my favorite part was reading about the pilgrimages. I have always been fascinated by different sacred places having made the trip to Rome to see the Vatican and hoping to someday visit Jerusalem.",0,562,11889,35,339.685714,2,Bibles are diff because o

In [70]:
df[clusters == 0] # Thinks they are different. I believe everyone should have access to all the books.

,body,score
8,"Being brought up as a Christian I was taught that the Bible was the only word of God. If one changed a single iota of the Bible you would be damned and surely go to hell. Now, however, I think about this differently. As long as your specific trait of Christianity do not add and delete books from the Bible as you see fit from time to time, that means that there is consistency in your Scriptures, then I find it acceptable that the books of the Bible are in adherence with your culture and situatedness.",9
9,"I wish that we could share all the books used in Christianity. I may sound prejudiced, but the Book of Mormon seems to be outside of the usual circle of Christianity. Mormons use the Old Testament and New but rely more on the Mormon scriptures. Not unlike the Muslims that have Old and New Testaments in their belief, but rely more on the Qur'an.",12
10,Question one is How do you define the tradition? It helps when we are studding together to have a common frame of reference. The LDS church Uses the King James Bible in English in part so we can more easily see the connections between the Bible and the Book of Mormon which was translated at a time when that was the common Bible and so uses that kind of scriptural language. The Authors of the Bible make massive use of quote and allusion to make their points and if you don't know what they are quoting you can miss 90% of the message. As an example most people don't know that the second most quoted book in the New testament after Isaiah is actually 1 Enoch\n\nCommon Books keep the discussion on what is meant as opposed to what is authoritative. Because we use the Book of Mormon a lot of time is used up speaking with other Christians on whether it is a legitimate source for doctrine.\n\nAs a resource here is [the online edition of the LDS scriptures][1]\n\n\n [1]: https://www.lds.org/scriptures?lang=eng,6
11,"I don't know. It probably helps with uniformity and communication but different writings will""speak"" to different people and so faith might be influenced by something outside the orthodoxy of your group - the problem, as I see it, is the attitude we have towards our own bible, usually that it is the only source of God's revelations, and our disregard ( in my case, ignorance) of everyone else's. Marshall",6
13,"I think different branches of christianity should be able to choose wich books they include in a bible because altough they share the same believe in Christ, the traditions have remarkable difference that go from liturgy to worship, art, and believes, therefore they hace one thing in common, that is to profess Jesus as saviour but have many differences from one another. For example, where I am from, Evangelical Christians and Roman Catholics see each other as a different religion itself.",3
26,"I would have to say that it would depend on how you view the bible. If you view it as a written by God or as the absolute word of God, then you'd probably think that everyone should use one standard version. But if you feel that the 10 commandments were they only ""word of God"" and that the rest is just stories, and moral values of a different time, then you might argue that while you should keep the 10 commandments, perhaps the rest might be in need of an update, or add on as time goes by.",1
28,"I think all those within a tradition should at least have ready access to all of the scriptures. I would be fascinated to read the Apocyphal books, and I think that I could learn a great deal from them. As for whether they should all be considered scripture, I am less certain. Definitely any work that is considered scripture by a large group of people is worthy of respect.",2
34,"I feel that it is important that people belonging to the same tradition share the same scriptures. It gives a starting point and a commonality that is necessary for people of the same tradition to be ""on the same page"". It is good to know and acknowledge that different Christian traditions do 

In [71]:
df[clusters == 1] # Believes that they are the same

,body,score
2,"I guess I would look at it as they roughly do share the heart of the same scriptures with some variation thrown in.\nWhen a passage is illuminated to me, that is what is is about.\n\n I don't find it helpful, tempting as it is, to get caught in analyzing(though as one participant has suggested, I do still have my head with me).",7
16,"I don't think it's important that all Christian bibles contain the same books. I think it's important that they do have some book in common though. I think this is a human characteristic, that we are strongly opinionated :)",13
19,Given the scriptural disagreements which gave rise to new forms of Christianity I doubt you could get a consensus on using the same books.,9
32,Is there a unifying feature within the books that are commonly accepted among all Christians? Perhaps we should delve deeper into context and actual book message.,0
36,"there are no rules, they can have different books if they want. Having the same books doesnt mean that poeple will interpret them the same way. Even if every branch had the same books they could take very different meaning from the same scripture",0
40,"Definitive lists are out for me. Just don't make any of the books heretical or banned so that I can read and discuss them openly, then decide for myself. \nI only realised last year when asked to organise a funeral that different Bibles had different books. So much to learn...so little time...",0
42,"I think not because not everyone has the opportunity to buy a Bible study whether or the different versions that exist today, while Jesus taught is taught I do not see problems",0
53,"I believe that all Christians should share the same Bible. Otherwise the definition of ""Christian"" becomes a bit difficult. If we have different Bibles then does it mean that some of us have a different set of beliefs based on the different texts?",0
61,I never knew there were so many different bibles. And I wonder how much of the New testament was the word of God?,0
62,Ideally there should be one set of what adherents consider as sacred texts.,0


In [72]:
df[clusters == 2] # They are different because of people but the message before people messed with it was the same.

,body,score
0,"I would like to start by noting that all the Christian traditions share many of the same books. The list provided overlap to a major extent. The differences often grew out of a difference in history. The Ethiopian bible, for instance, grew out of the circumstance that they are one of the oldest Christian communities and were quite isolated from other branches of the tradition for a long time. It would seem unreasonable to ask them to give up portions of their scripture in order to be in synch with most other Christians who differ greatly among themselves.\nThe Bible has never seemed to me to be one thing. It is a collection of books. Some of those books are reproduced in versions meant for children, others are turned into Sunday morning cartoon versions. The various books have been translated in various ways by people with particular agendas. The Good News bible, for example, is meant for people who may struggle with some of the standard vocabulary,.\nI live in a part of the world that seems to ""worship"" the King James version of the bible, which seems quite unreasonable as his translators did not have the skills in Greek and Aramaic that have led scholars to some of their interesting revisions.\nI for one would consider the Bible to be a work in progress, that has variations that have meaning to various groups at various times.",31
5,"From an abstract standpoint, I think it is important that members of the same faith share the same scripture. In the introductory video scripture is defined as literature with unique authority; I think if two groups believe that they share the same faith but disagree on the source of divine authority there are obviously going to be conflicts. One of the reasons I phrased my answer in that way is that many individuals and denominations that consider themselves christians do not consider other denominations that do not share their bible christian. To me, the question kind of begs the question of wether the different faith traditions we are studying are the same religion at all, based purely on some degree of shared scripture. My question here is similar to one I was trying to answer in my original post; at what point does scripture become so different it is no longer christian at all? \n\nIn the concrete case of christianity, I think we need to look at the history behind the differentiation of the bible, as other comments have suggested. My understanding is that the bible was originally a set of books that circulated individually among early christian communities. I think that your decision about what books to use is a decision about how legitimate the editing process that brought us those books was. You could decide that all revelation circulated as a book of the bible in the years after jesus' death was divine, and see the dead sea scrolls as legitimate in the same way the letters of Paul are. Or you could decide that the church authorities who created the holy bible were divinely inspired, as you could with any other codification of the books of the bible. With mormonism this faith in divine inspiration of scripture goes down to the 1800's. I think this is one of the conflicts inherent in religion: everyone of the same faith should follow the same scripture, but scripture, (and especially its interpretation), must be an ongoing, living thing, which inevitably leads to differences in scripture and interpretation across cultures.",6
17,"I think scriptures are the same in all Christian traditions. Christianity evolved over 2000 years, while Judaism over longer period, without central authority and separated by geographical and political boundaries. For example, Christians of Roman and Persian Empires had limited contacts as those empires were hostile to each other. It is natural for slight differences to evolve. Differences in book names, order of books, presence or absence of some paragraphs or inclusion of Apocrypha do not make scriptures' contents different. On opposite it is i

In [73]:
df[clusters == 3] # No, not important to use the same scripture. Differences are okay!

,body,score
3,"It is important for members of the same traditions to share the same scriptures but it is not completely necessary. All Christians share the most ""important"" books of the Bible. Most of the additional books found in other Bibles are generally less important the the Gospels and letters of the New Testament, Psalms, and the Torah. Key scriptures are necessary to be shared by a tradition but the addition of other books does not break the deal.",9
6,I see it as the same religious tradition if the majority of the texts are the same. Including differing books does raise barriers between practitioners it also demonstrates that there are some very deep divisions of interpretation. There are inherent contradictions within each chosen set of Christian books so any grouping is problematical.,2
7,The bible is a complation of books. On specific moments in time and for specific reasons some people decided to well/not include some of these books in 'their' bible. I feel it is very interesting to study the process of in/exclusion. I am a historian and not a theologian! Furthermore: the different bibles do show how difficult it is to maintain the position that the whole bible is literally the word of god.,11
14,There should be a Bible that is common to all. The teachings of the Christian Church and the live of Christians rely on the Bible as their blueprint..What the Church teaches must always be proven from the Bible,12
15,"I believe it is important for those who belong to the same religious community to have a common point of reference. I feel more relaxed about different traditions within the same religion enjoying modest variations, which reflect their historical experience. For the purpose of good relations and mutual respect, we should acquaint ourselves with these variations.",6
18,"I think people belonging to the same tradition should share the same Scriptures because they share the same faith, and the Scriptures would be their guidance for living.",5
21,"I think that using a certain system of books restricts the amount of confusion that may arise from many scriptures.\n\nOn the other hand I use several books that other books in the Christian canon quotes from, but were left out in 325 a.d. when king Constantine brought all of the heads of the many groups that had formed. They all debated over doctrinal issues such as Trinity vs. Oneness, and which books that should be canonized. Two of the book which are founded in my library are the book of Enoch, the book of Jasher.",9
23,"I would like all included, to increase overall knowledge of the ""Holy Scriptures"". I am sure debate was held many times over as to what was necessary for laymen to read. It was probably based on the direction the clergy wanted to use the text to create their own divine base.",15
24,I think that should be more tolerance between Christian churches in accepting the different Bible texts I have Protestant relatives that have very little understanding why a Catholic bible has more books. tHey are all teaching the word of God .Also there are so many versions. tHis person has a bible with his name attached. this seems reduntant,14
25,"While I believe that a shared scripture among a religious tradition is good for establishing a sense of continuity and oneness, I don't think it's important nor necessary for them to all settle on one definitive canonical list. However, I do recognize that this is the opinion of someone viewing this as an outsider and from a nonsectarian study of religion, and that a practicing Christian viewing this from the standpoint of devotional expression may see things differently.",3


In [74]:
df[clusters == 4] # Equivocating both sides

,body,score
1,"SHOULD PEOPLE IN THE SAME RELIGIOUS TRADITION SHARE THE SAME SCRIPTURES?\n\nMy short answer to this question is: Yes. \n\nFor clarity and ease of communication, it is important that all members of a religious tradition share the same version of the Bible. \n\nMy longer answer to the question would be that life never remains constant therefore it is critical that members of a given tradition seek ways to mold their religious beliefs to a changing world. Consulting other bibles might be one way of doing that as well as consulting non-biblical sources such as biblical archeology. Violence and schisms within a religion occur when people refuse to consider modifying a belief system even in the face of overwhelming change within their environment. \n\nI think we are seeing this at present in the USA. We have groups who insist that politicians support and enforce very narrow religious views even when a majority of citizens do not support those views. Contraception and Women's Reproductive Rights is just one example; there are many, many more.",20
4,"As it has been noticed here, the Bible is not a book but a collection of books. It is important to understand why some book were included or excluded from the different versions. It involves complex consideration of theological concepts and variations in practices, historical, political, cultural and other circumstances. It might be easier for the international Christian community if the Bible is one, unified text. However, I think that is impossible to achieve. It has to be some reason why some texts are canonical for one group of Christians but excluded from other versions of the Bible. You cannot make one group of Christians to abandon their beliefs in validity of certain books excluded from other versions, and you can hardly make other groups to accept other texts in addition to what they already have because those texts were not included to their version of the Bible before for some reason. This reason may be already outdated (like some political circumstances, for instance) or may be still valid (as some canonical differences in religious practices or liturgy). General practitioners may not be aware of those reasons and differences, but I'm sure the clergy do. \nI was raised in Ukraine in Eastern Orthodox Christian tradition. At some point I found it interesting to find online and read some books not included into the Orthodox Bible. But I have never done more research on why they are not included. I hope to get more answers on this questions about scriptural variations from this course.",5
12,"Before embarking on this course and Professor Moore's previous course, I would have answered that I believe the same faith traditions should share the same scriptures. Now, however, I realize this is a sign of internal diversity within the same religion. In addition, reading the Resources for Additional Learning from Day 1 offered a segment from Douglas Jacobsen's ""The World's Christians"" which explains that Christianity is united by a number of factors - the idea that Jesus is human and divine, the Trinity, that salvation is a gift from God, the opportunity for weekly worship, baptism as a rite of initiation, and the Eucharist - although not everyone believes in transubstantiation. In addition, as we learned today, ""all Christian Bibles"" are based on the Tanakh. Thus, although I am still adjusting to the idea that Christianity contains so many different Bibles, I do not believe that the same religions must share the exact same scriptures, for they share other common factors that place them under the umbrella of Christian or whatever other religion we may discuss.",8
20,"It would be ideal to have the same information to facilitate common understanding but very likely unrealistic. With historical and regional differences, varied translations and widely diverse interpretations the variability is easily understood.\n\nIt is unfortunate that we are so unaware of scriptural diff